In [4]:
# CELL 1: Install
import sys, subprocess
pkgs = ["copernicusmarine", "pandas", "ipywidgets", "xarray", "netCDF4"]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + pkgs + ["--quiet"])
print("Done!")

Done!


In [5]:
# CELL 2: Imports
import os
import copernicusmarine
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from IPython.display import display, clear_output
from pathlib import Path

DATASET = "cmems_obs-wind_glo_phy_nrt_l4_0.125deg_PT1H"
VARS = ["stress_curl", "stress_curl_bias", "stress_curl_dv", 
        "stress_divergence", "stress_divergence_bias", "stress_divergence_dv"]

print(f"copernicusmarine version: {copernicusmarine.__version__}")
print(f"Dataset: {DATASET}")

copernicusmarine version: 2.3.0
Dataset: cmems_obs-wind_glo_phy_nrt_l4_0.125deg_PT1H


In [6]:
# CELL 3: Interface

def parse_dates(filepath):
    dates = []
    skip = ('#', '=', '-', 'lista', 'total', 'date', 'unique', 'list')
    with open(filepath) as f:
        for line in f:
            line = line.split('#')[0].strip()
            if not line or any(line.lower().startswith(s) for s in skip):
                continue
            try:
                dates.append(pd.to_datetime(line))
            except:
                pass
    return dates

def download_wind(lat, lon, mode, date_in, outdir, user, pwd, prog=None):
    # Parse dates
    dates = []
    if mode == 'Single Date':
        dates = [pd.to_datetime(date_in)]
    elif mode == 'Date Range':
        dates = pd.date_range(date_in[0], date_in[1], freq='D').tolist()
    else:
        if not os.path.exists(date_in):
            print(f"File not found: {date_in}")
            return
        dates = parse_dates(date_in)
    
    if not dates:
        print("No dates found")
        return
    
    dates = sorted(dates)
    print(f"{len(dates)} dates: {dates[0].date()} to {dates[-1].date()}")
    
    if not user or not pwd:
        print("Need user/password")
        return
    
    # Login - check if creds exist first
    cred = Path.home() / '.copernicusmarine' / '.copernicusmarine-credentials'
    if cred.exists():
        print("Using saved credentials")
    else:
        print(f"Logging in as {user}...")
        copernicusmarine.login(username=user, password=pwd)
    
    # Output dir
    outpath = Path(outdir)
    outpath.mkdir(parents=True, exist_ok=True)
    print(f"Output: {outpath}")
    
    # Check existing
    todo = []
    skip = 0
    for d in dates:
        f = outpath / f"cmems_wind_{d.strftime('%Y%m%d')}.nc"
        if f.exists():
            skip += 1
        else:
            todo.append(d)
    
    if skip:
        print(f"Skipping {skip} existing")
    if not todo:
        print("All done!")
        return
    
    print(f"\nDownloading {len(todo)} files...")
    print(f"Region: lat[{lat[0]}:{lat[1]}] lon[{lon[0]}:{lon[1]}]")
    
    if prog:
        prog.max = len(todo)
        prog.value = 0
    
    ok = 0
    for i, d in enumerate(todo):
        try:
            copernicusmarine.subset(
                dataset_id=DATASET,
                variables=VARS,
                minimum_longitude=lon[0],
                maximum_longitude=lon[1],
                minimum_latitude=lat[0],
                maximum_latitude=lat[1],
                start_datetime=d.strftime('%Y-%m-%dT00:00:00'),
                end_datetime=d.strftime('%Y-%m-%dT23:59:59'),
                output_filename=f"cmems_wind_{d.strftime('%Y%m%d')}.nc",
                output_directory=str(outpath)
            )
            print(f"  OK {d.date()}")
            ok += 1
        except Exception as e:
            print(f"  FAIL {d.date()}: {str(e)[:50]}")
        if prog:
            prog.value = i + 1
    
    print(f"\nDone: {ok} downloaded, {skip} skipped")

# Folder browser
class FolderBrowser:
    def __init__(self, start='.'):
        self.cur = Path(start).resolve()
        self.sel = self.cur
        self.html = widgets.HTML(f"<code>{self.cur}</code>")
        self.dd = widgets.Select(options=self._list(), layout=widgets.Layout(width='100%', height='100px'))
        self.b_up = widgets.Button(description='Up', layout=widgets.Layout(width='60px'))
        self.b_in = widgets.Button(description='Enter', layout=widgets.Layout(width='70px'))
        self.b_sel = widgets.Button(description='Select', button_style='success', layout=widgets.Layout(width='70px'))
        self.txt = widgets.Text(placeholder='new folder', layout=widgets.Layout(width='120px'))
        self.b_new = widgets.Button(description='+', layout=widgets.Layout(width='40px'))
        self.selhtml = widgets.HTML(f"Selected: <code>{self.sel}</code>")
        self.b_up.on_click(lambda b: self._up())
        self.b_in.on_click(lambda b: self._enter())
        self.b_sel.on_click(lambda b: self._select())
        self.b_new.on_click(lambda b: self._create())
        self.w = widgets.VBox([widgets.HTML("<b>Output Folder</b>"), self.html, self.dd,
                               widgets.HBox([self.b_up, self.b_in, self.b_sel, self.txt, self.b_new]), self.selhtml])
    def _list(self):
        try:
            return ['.'] + [x.name for x in sorted(self.cur.iterdir()) if x.is_dir() and not x.name.startswith('.')]
        except:
            return ['.']
    def _refresh(self):
        self.html.value = f"<code>{self.cur}</code>"
        self.dd.options = self._list()
    def _up(self):
        if self.cur.parent != self.cur:
            self.cur = self.cur.parent
            self._refresh()
    def _enter(self):
        if self.dd.value and self.dd.value != '.':
            p = self.cur / self.dd.value
            if p.is_dir():
                self.cur = p
                self._refresh()
    def _select(self):
        self.sel = self.cur
        self.selhtml.value = f"Selected: <code>{self.sel}</code>"
    def _create(self):
        n = self.txt.value.strip()
        if n:
            p = self.cur / n
            p.mkdir(parents=True, exist_ok=True)
            self.cur = p
            self.sel = p
            self.txt.value = ''
            self._refresh()
            self.selhtml.value = f"Created: <code>{self.sel}</code>"
    def path(self):
        return str(self.sel)

# File browser
class FileBrowser:
    def __init__(self, start='.', ext=None):
        self.cur = Path(start).resolve()
        self.sel = None
        self.ext = ext or ['.txt', '.csv']
        self.html = widgets.HTML(f"<code>{self.cur}</code>")
        self.dd = widgets.Select(options=self._list(), layout=widgets.Layout(width='100%', height='120px'))
        self.b_up = widgets.Button(description='Up', layout=widgets.Layout(width='60px'))
        self.b_in = widgets.Button(description='Enter', layout=widgets.Layout(width='70px'))
        self.b_sel = widgets.Button(description='Select File', button_style='success', layout=widgets.Layout(width='90px'))
        self.selhtml = widgets.HTML("<i>No file</i>")
        self.b_up.on_click(lambda b: self._up())
        self.b_in.on_click(lambda b: self._enter())
        self.b_sel.on_click(lambda b: self._select())
        self.w = widgets.VBox([widgets.HTML("<b>Date List File</b>"), self.html, self.dd,
                               widgets.HBox([self.b_up, self.b_in, self.b_sel]), self.selhtml])
    def _list(self):
        try:
            items = []
            for x in sorted(self.cur.iterdir()):
                if x.name.startswith('.'): continue
                if x.is_dir():
                    items.append(f"[dir] {x.name}")
                elif x.suffix.lower() in self.ext:
                    items.append(x.name)
            return items if items else ['(empty)']
        except:
            return ['(err)']
    def _refresh(self):
        self.html.value = f"<code>{self.cur}</code>"
        self.dd.options = self._list()
    def _up(self):
        if self.cur.parent != self.cur:
            self.cur = self.cur.parent
            self._refresh()
    def _enter(self):
        v = self.dd.value
        if v and v.startswith('[dir]'):
            p = self.cur / v.replace('[dir] ', '')
            if p.is_dir():
                self.cur = p
                self._refresh()
    def _select(self):
        v = self.dd.value
        if v and not v.startswith('[') and not v.startswith('('):
            self.sel = self.cur / v
            self.selhtml.value = f"<code>{self.sel}</code>"
    def file(self):
        return str(self.sel) if self.sel else None

# UI
fb = FolderBrowser('.')
flb = FileBrowser('.', ['.txt', '.csv', '.dat'])

w_user = widgets.Text(description='User:', layout=widgets.Layout(width='35%'))
w_pwd = widgets.Password(description='Password:', layout=widgets.Layout(width='35%'))
w_lat = widgets.FloatRangeSlider(value=[30,50], min=-90, max=90, step=0.5, description='Lat:', layout=widgets.Layout(width='50%'), continuous_update=False)
w_lon = widgets.FloatRangeSlider(value=[-80,-60], min=-180, max=180, step=0.5, description='Lon:', layout=widgets.Layout(width='50%'), continuous_update=False)
w_mode = widgets.Dropdown(options=['Single Date', 'Date Range', 'Date List File'], value='Single Date', description='Mode:')
w_single = widgets.DatePicker(description='Date:', value=datetime(2024,3,30))
w_start = widgets.DatePicker(description='Start:', value=datetime(2024,3,30))
w_end = widgets.DatePicker(description='End:', value=datetime(2024,4,1))
w_datebox = widgets.VBox([w_single])

def mode_change(c):
    if c['new'] == 'Single Date':
        w_datebox.children = [w_single]
    elif c['new'] == 'Date Range':
        w_datebox.children = [widgets.HBox([w_start, w_end])]
    else:
        w_datebox.children = [flb.w]
w_mode.observe(mode_change, 'value')

w_prog = widgets.IntProgress(min=0, max=1, description='Progress:', layout=widgets.Layout(width='50%'))
w_btn = widgets.Button(description='DOWNLOAD', button_style='success', layout=widgets.Layout(width='100%', height='40px'))
w_log = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', max_height='300px', overflow='auto'))

def click(b):
    with w_log:
        clear_output()
        mode = w_mode.value
        if mode == 'Single Date':
            din = w_single.value
        elif mode == 'Date Range':
            din = (w_start.value, w_end.value)
        else:
            din = flb.file()
            if not din:
                print("Select a file first!")
                return
        download_wind(w_lat.value, w_lon.value, mode, din, fb.path(), w_user.value, w_pwd.value, w_prog)

w_btn.on_click(click)

display(widgets.VBox([
    widgets.HTML("<h2>CMEMS Wind Stress Downloader</h2>"),
    widgets.HBox([w_user, w_pwd]),
    fb.w,
    widgets.HTML("<b>Region</b>"), w_lat, w_lon,
    widgets.HTML("<b>Dates</b>"), w_mode, w_datebox,
    w_btn, w_prog,
    widgets.HTML("<b>Log:</b>"), w_log
]))